In [18]:
import warnings
warnings.filterwarnings('ignore')

## 빅데이터 분석기사 실기 
---
### 1. 실습 A-Z : Motor Trend Car Road Tests
1. 문제 
- mtcars 데이터셋을 변형시킨 데이터 

### 2. 사전 준비

In [1]:
# 2. 데이터 형식 및 내용 

### 처음 시험 시작하면 아래와 같거나 비슷한 수준의 예시 코드가 주어짐

In [4]:
# 데이터 파일 읽기 예제 
import pandas as pd

X_test = pd.read_csv('/Users/hayeon/Downloads/mpg_X_test.csv')
X_train = pd.read_csv('/Users/hayeon/Downloads/mpg_X_train.csv')
y_train = pd.read_csv('/Users/hayeon/Downloads/mpg_y_train.csv')

---

In [5]:
import pandas as pd
import numpy as np

In [10]:
X_test = pd.read_csv('/Users/hayeon/Downloads/mpg_X_test.csv')
X_train = pd.read_csv('/Users/hayeon/Downloads/mpg_X_train.csv')
y_train = pd.read_csv('/Users/hayeon/Downloads/mpg_y_train.csv')

### 3. 데이터 전처리

In [7]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          278 non-null    object 
 1   mpg           278 non-null    float64
 2   cylinders     278 non-null    int64  
 3   displacement  278 non-null    float64
 4   horsepower    274 non-null    float64
 5   weight        278 non-null    int64  
 6   acceleration  278 non-null    float64
 7   model_year    278 non-null    int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 17.5+ KB
None


### imputer 
- 특정 값을 대체할 때 사용하는 모듈

In [11]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean') # nan에 해당하는 값을 바꿔준다
X_train[['horsepower']] = imputer.fit_transform(X_train[['horsepower']])
X_test[['horsepower']] = imputer.fit_transform(X_test[['horsepower']])

In [12]:
print(X_train.describe())

              mpg   cylinders  displacement  horsepower       weight  \
count  278.000000  278.000000    278.000000  278.000000   278.000000   
mean    23.732734    5.374101    189.994604  103.383212  2948.464029   
std      7.647295    1.677084    105.471423   38.695458   862.949746   
min     10.000000    3.000000     68.000000   46.000000  1613.000000   
25%     18.000000    4.000000     98.000000   75.000000  2206.250000   
50%     23.000000    4.000000    140.500000   90.500000  2737.500000   
75%     29.000000    6.000000    258.000000  118.750000  3560.000000   
max     46.600000    8.000000    455.000000  230.000000  5140.000000   

       acceleration  model_year  
count    278.000000  278.000000  
mean      15.580216   76.057554  
std        2.745907    3.605591  
min        8.000000   70.000000  
25%       14.000000   73.000000  
50%       15.500000   76.000000  
75%       17.000000   79.000000  
max       24.800000   82.000000  


In [13]:
COL_DEL = ['name']
COL_NUM = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year']
COL_CAT = []
COL_Y = ['isUSA']

X_train = X_train.iloc[:, 1:]
X_test = X_test.iloc[:, 1:]

In [14]:
X_train

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
0,31.0,4,112.0,85.0,2575,16.2,82
1,13.0,8,400.0,175.0,5140,12.0,71
2,37.0,4,91.0,68.0,2025,18.2,82
3,12.0,8,350.0,180.0,4499,12.5,73
4,19.0,4,120.0,88.0,3270,21.9,76
...,...,...,...,...,...,...,...
273,36.1,4,91.0,60.0,1800,16.4,78
274,30.0,4,97.0,67.0,1985,16.4,77
275,33.5,4,98.0,83.0,2075,15.9,77
276,18.0,6,171.0,97.0,2984,14.5,75


### 4. 데이터 모형 구축

In [40]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3)

In [41]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr[COL_NUM])

X_tr[COL_NUM] = scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.transform(X_test[COL_NUM])

In [42]:
from sklearn.neighbors import KNeighborsClassifier

modelKNN = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
modelKNN.fit(X_tr, y_tr.values.ravel())

from sklearn.tree import DecisionTreeClassifier
modelDT = DecisionTreeClassifier(max_depth=10)
modelDT.fit(X_tr, y_tr)

DecisionTreeClassifier(max_depth=10)

### 5. 데이터 모형 평가

In [43]:
y_val_pred = modelKNN.predict(X_val)

y_val_pred_probaKNN = modelKNN.predict_proba(X_val)
y_val_pred_probaDT = modelDT.predict_proba(X_val)

In [44]:
from sklearn.metrics import roc_auc_score

scoreKNN = roc_auc_score(y_val, y_val_pred_probaKNN[:, 1])
scoreDT = roc_auc_score(y_val, y_val_pred_probaDT[:, 1])

print(scoreKNN, scoreDT)

0.8625 0.8045454545454546


In [45]:
# best model selection 
best_model = None
best_score = 0

for i in range(2, 10):
    model = KNeighborsClassifier(n_neighbors=i, metric='euclidean')
    model.fit(X_tr, y_tr.values.ravel())
    y_val_pred_proba = model.predict_proba(X_val)
    score = roc_auc_score(y_val, y_val_pred_proba[:, 1])
    print(i, "개의 이웃 확인 : ", score)
    if best_score <= score:
        best_model = model

2 개의 이웃 확인 :  0.8696022727272728
3 개의 이웃 확인 :  0.85
4 개의 이웃 확인 :  0.8630681818181819
5 개의 이웃 확인 :  0.8625
6 개의 이웃 확인 :  0.8823863636363637
7 개의 이웃 확인 :  0.9017045454545455
8 개의 이웃 확인 :  0.8852272727272728
9 개의 이웃 확인 :  0.8886363636363637


### 7개의 군집일 때 가장 성능이 좋음
### 이제 제출을 위해 문제에서 요구하는 제출물 만들기

In [46]:
print(best_model.predict_proba(X_test))

[[0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.33333333 0.66666667]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.33333333 0.66666667]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]
 [0.44444444 0.55555556]


In [47]:
# 미국에 해당하는 경우가 1이기 때문에 두번째 열에 해당하는게 우리가 추출해야 하는 확률 값 
pred = best_model.predict_proba(X_test)[:, 1]
print(pred)

[0.55555556 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556
 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556
 0.66666667 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556
 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556 0.66666667
 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556
 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556
 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556
 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556
 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556
 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556
 0.55555556 0.55555556 0.66666667 0.55555556 0.55555556 0.55555556
 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556
 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556
 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556
 0.55555556 0.55555556 0.55555556 0.55555556 0.55555556 0.5555

In [49]:
pd.DataFrame({'isUSA' : pred}).to_csv('003000000.csv', index=False)

## Tip 
- 성능 떨어지는 단순 기본 모형이라도 빠르게 완성하고, 시간적 여유가 있을 때 다양한 모형을 실험해보거나 파라미터 튜닝하는 등.. 은 나중에!